In [15]:
import numpy as np
import pandas as pd
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.core.problem import ElementwiseProblem
from pymoo.optimize import minimize
from pymoo.visualization.scatter import Scatter

# Generate synthetic data
n_vehicles = 20
n_sites = 50
np.random.seed(42)

vehicles_data = pd.DataFrame({
    'nom': [f'vehicule_{i}' for i in range(1, n_vehicles + 1)],
    'vehicule_lat': np.random.uniform(32.86, 32.91, n_vehicles),
    'vehicule_lng': np.random.uniform(-6.94, -6.88, n_vehicles),
    'vehicule_type': np.random.randint(1, 4, n_vehicles),
    'capacite': np.random.randint(1, 4, n_vehicles)
})

sites_data = pd.DataFrame({
    'site_name': [f'site_{i}' for i in range(1, n_sites + 1)],
    'site_lat': np.random.uniform(32.86, 32.91, n_sites),
    'site_lng': np.random.uniform(-6.94, -6.88, n_sites),
    'niveau_incident': np.random.randint(1, 4, n_sites)
})

# Calculate distance matrix
distances = np.zeros((n_vehicles, n_sites))
for i, vehicle in vehicles_data.iterrows():
    for j, site in sites_data.iterrows():
        distances[i, j] = np.sqrt((vehicle['vehicule_lat'] - site['site_lat'])**2 +
                                  (vehicle['vehicule_lng'] - site['site_lng'])**2)

# Problem definition
class EmergencyVehicleProblem(ElementwiseProblem):
    def __init__(self):
        super().__init__(n_var=n * m, n_obj=1, n_constr=m + 2 * n * m, xl=0, xu=1)
 
    def _evaluate(self, x, out, *args, **kwargs):
        X = x.reshape(n, m)
        t = a * d + b * v
        response_time = np.sum(t * X)
        out["F"] = response_time
 
        G = np.zeros(m + 2 * n * m)
 
        # First m constraints
        for j in range(m):
            G[j] = L[j] - np.sum(C * X[:, j])
 
        # Next n * m constraints for x_{ij} <= y_i
        for i in range(n):
            for j in range(m):
                G[m + i * m + j] = X[i, j] - 1
 
        # Last n * m constraints for y_i >= x_{ij}
        for i in range(n):
            for j in range(m):
                G[m + n * m + i * m + j] = 1 - X[i, j]
 
        out["G"] = G

# Initialize and run the algorithm
algorithm = NSGA2(pop_size=100)

problem = EmergencyVehicleProblem()

res = minimize(
    problem,
    algorithm,
    seed=1,
    termination=('n_gen', 100),
    verbose=True,
)

# Visualize and print the best solution
# Scatter().add(res.F).show()

# best_solution = res.X[np.argmin(res.F)].reshape(n_vehicles, n_sites)
print("Best solution found:")
print(res.F)


n_gen  |  n_eval  | n_nds  |     cv_min    |     cv_avg    |      eps      |   indicator  
     1 |      100 |      1 |  2.7583700645 |  4.5220265026 |             - |             -
     2 |      200 |      1 |  1.9385424532 |  3.5312343941 |             - |             -
     3 |      300 |      1 |  1.3522124622 |  2.8441625154 |             - |             -
     4 |      400 |      1 |  1.1076676642 |  2.3124099197 |             - |             -
     5 |      500 |      1 |  0.7207127934 |  1.7794549177 |             - |             -
     6 |      600 |      1 |  0.6855963878 |  1.4189749805 |             - |             -
     7 |      700 |      1 |  0.4760173925 |  1.1252345226 |             - |             -
     8 |      800 |      1 |  0.4668678471 |  0.8824198360 |             - |             -
     9 |      900 |      1 |  0.2967666176 |  0.6495989654 |             - |             -
    10 |     1000 |      1 |  0.2599658038 |  0.4982518755 |             - |             -